<a href="https://colab.research.google.com/github/InryeolChoi/nlp_with_GPT_and_bert/blob/main/NLP_with_Bert_and_GPT%3D8_3%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install packages
< Eng >
* Install packages with pip

< Kor >
* pip로 ratsnlp 패키지를 설치합니다.

In [ ]:
!pip install ratsnlp

# 모델 로딩 (model loading)
* 프리트레인한 GPT2 모델과 토크나이저를 읽어 들입니다.
* load the pretrained GPT2 model and tokenizer

In [ ]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)
model.eval()

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

## 프롬프트 준비 (preparing prompt)
* 언어모델에 넣을 프롬프트를 준비합니다.
* prepare the prompt to put in language model 

In [ ]:
input_ids = tokenizer.encode("안녕하세요", return_tensors="pt")

## Greedy Search

< Kor >
* 다음 단어 확률 분포에서 최대 확률을 내는 단어들을 리턴합니다. 
* 여러 번 수행하더라도 생성 결과가 바뀌지 않습니다 (`do_sample=False`).
* `max_length`는 생성 최대 길이이며 이보다 길거나, 짧더라도 EOD 토큰 등 스페셜 토큰이 나타나면 생성을 중단합니다. 
* `min_length`는 생성 최소 길이이며 이보다 짧은 구간에서 EOD 등 스페셜 토큰이 등장해 생성이 중단될 경우 해당 토큰이 나올 확률을 0으로 수정하여 문장 생성이 종료되지 않도록 강제합니다.

< Eng >
* Returns the words with the highest probability from the next word probability distribution.
* Even if performed multiple times, the result does not change (do_sample=False).
* `max_length` is the maximum length for generation; generation stops if it exceeds this length or if a special token like EOD appears, even if it's shorter.
* `min_length` is the minimum length for generation; if a special token like EOD appears in a shorter span than this, its probability is adjusted to zero to prevent the end of sentence generation.

In [ ]:
import torch
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## Beam Search 

< Kor >
* Beam Search는 다음 단어 확률 분포에서 `num_beams`만큼의 경우의 수를 남겨가면서 문장을 생성합니다.  
* Beam search는 Greedy search보다 계산량이 많지만 좀 더 확률값이 높은 문장을 생성할 수 있습니다.

< Eng >
* Beam Search generates sentences by retaining a number of possibilities equal to num_beams from the next word probability distribution.
* Beam search requires more computation than Greedy search, but it can generate sentences with higher probability values.

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        num_beams=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그


* `num_beams=1`로 설정하면 정확히 Greedy search와 동일하게 작동합니다.
* It works same as Greedy Search when setting `num_beams=1`

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        num_beams=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## Reducing Repetition (반복 줄이기)

### 반복되는 n-gram 사이즈를 지정하기 (Specifying the Size of Repeating N-grams)

< Kor >
* 위의 예시를 보면 `"그럼, 그건 뭐예요?"`이 반복됩니다. 
* 이를 아래와 같이 지정해 반복을 방지합니다. 
* 3개 이상의 토큰이 반복될 경우 해당 3-gram 등장 확률을 0으로 만들어 생성 결과에서 배제합니다.

< Eng >
* In the example above, "그럼, 그건 뭐예요?" is repeated.
* To prevent this repetition, it can be specified as follows:
* If a sequence of 3 or more tokens is repeated, the probability of that 3-gram appearing is set to zero, excluding it from the generation results.

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        no_repeat_ngram_size=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?" 하고 나는 물었다.
"그건 뭐죠?" 나는 물었다.
나는 대답하지 않았다.
"그런데 왜 그걸 물어요? 그건 무슨 뜻이에요?


### repetition penalty

< kor >
* repetition penalty로 반복을 통제할 수도 있습니다. 
* 다음과 같이 실행하면 되며 그 범위는 1 이상의 값을 가져야 합니다. 
* 1이라면 아무런 패널티를 적용하지 않는게 됩니다.

< eng >
* Repetition can also be controlled using a repetition penalty.
* It can be applied as follows, and its value must be greater than 1.
* A value of 1 means that no penalty is applied.

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



`repetition_penalty` 값이 클 수록 패널티가 세게 적용됩니다.

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"아니요, 저는요."
"그럼, 그건 무슨 말씀이신지요?"
"그럼, 그건 뭐예요?"



In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.2,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요."
"그래서 오늘은 제가 할 수 있는 일이 무엇인지 말해 보겠습니다."
"이제


In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요. 그리고 제가 할 수 있는 일은 아무것도 없어요.
이제 그만 돌아가고 싶어요.
제가 하는 일이 무엇


## top-k sampling

< kor >
* 지금까지는 생성을 반복하더라도 그 결과가 동일한 샘플링 방식을 살펴봤습니다. 
* top-k sampling은 다음 단어를 뽑을 때 확률값 기준 가장 큰 k개 가운데 하나를 선택하는 기법입니다. 
* 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, k개 안에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있습니다.
* 따라서 top-k sampling은 매 시행 때마다 생성 결과가 달라집니다. k는 1 이상의 값을 지녀야 합니다.

< eng >
* We have looked at sampling methods where the results remain the same even if generation is repeated.
* Top-k sampling is a technique that selects one of the k words with the highest probability values as the next word.
* While words with higher probability values are more likely to be chosen as the next word, any word within the top k can be selected, even if it has a lower probability.
* Therefore, with top-k sampling, the results vary with each execution. The value of k must be greater than or equal to 1.

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?'라고 응원을 보냈습니다.
하지만 '모르는 척 말하라', '그렇게 해라, 고마워'라는 대답이 돌아왔습니다.
이때가 7회.
모른 척 행동하던 한화가 흔들리는 가운데, 한화는 승리의


k=1일 경우 Greedy search와 동일합니다. 

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## top-k sampling + temperature scaling

* top-k sampling은 temperature scaling과 동시에 적용할 수 있습니다. 
* Top-k sampling can be applied simultaneously with temperature scaling.

(1) 그 값에 따라 다음과 같은 효과가 납니다.  
1. t가 0에 가까워질 수록 토큰 분포가 sharp해진다  
2. 1등 토큰이 뽑힐 확률이 그만큼 높아진다  
3. do_sample=True이지만 사실상 greedy decoding이 된다

(1) Depending on the value of temperature (t), the following effects occur:  
1. As t approaches 0, the token distribution becomes sharper.  
2. The probability of selecting the top-ranked token increases.  
3. Although do_sample=True, it essentially becomes greedy decoding.  

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



(2) part 2  
< kor >
1. t=1이라면 모델 출력 분포를 그대로 사용한다.
2. 하지만 샘플링 방식을 사용하기 때문에 생성할 때마다 다른 문장이 나온다.

< eng >
1. If t=1, the model uses the output distribution as it is.
2. However, because a sampling method is used, a different sentence is generated each time.

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요! ^^ 
<unk>http://www.1368.co.kr/business/delivery.
#우한폐렴 #우한폐렴 #쯔쯔가이쯔쯔


(3) t를 키울수록 토큰 분포가 uniform해진다 > 사실상 uniform sampling이 된다, 생성 품질이 악화할 가능성이 높아진다

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=100000000.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요' 또는 '불안해진 마음을 좀 풀기 위한 시간 가질 겁지다'는 메시로 위드 맵 오큘리(위버스 애슬리트)나 미션과 더불어 직접 만나야 하기 때 가장 기억 난답다고 밝혔었다. 고객


## top-p sampling

top-p sampling은 다음 단어를 뽑을 때 누적 확률값이 p 이하인 단어들 가운데 하나를 선택하는 기법입니다. 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, 누적 확률값 p 이하에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있습니다. 따라서 top-p sampling은 매 시행 때마다 생성 결과가 달라집니다. p는 확률이기 때문에 0~1 사이의 값을 지녀야 합니다. p가 1이라면 어휘 집합에 있는 모든 단어를 대상으로 샘플링하기 때문에 top-p sampling 효과가 사라집니다. p가 1보다 약간 작다면 확률값이 낮은 일부 단어들을 다음 단어 후보에서 제거해 생성 품질을 높입니다. 

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요...?
뭐가 진짜 그렇게 맛있죠?!!!
김치!! 김치와 함께 먹어야죠!!!!
<unk><unk> (사진찍는거 ) (<unk><unk><unk><unk><unk><unk><unk><unk>)
#다


p가 0에 가까울 경우 Greedy search와 비슷해 집니다.

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## 통합 적용

저희가 실습에 사용하고 있는 허깅페이스(huggingface) 라이브러리의 구현상 적용 순서는 다음과 같습니다.  
(The sequence of our settings regaridng huggingface libraries is as follows)

- _get_logits_processor
  - RepetitionPenalty
  - NoRepeatNGramLogits
  - MinLengthLogits
- _get_logits_warper
  - TemperatureLogits
  - TopKLogits
  - TopPLogits

유효한 설정들을 종합 적용해 문장을 생성하는 코드는 다음과 같습니다. 샘플링(top-k, top-p)을 적용하기 때문에 시행 때마다 다른 문장이 생성됩니다.  
The code to generate sentences by comprehensively applying valid settings is as follows.  
different sentences are generated each time as samping(top-k, top-p) applies

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3,
        temperature=0.9,
        top_k=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))